In [ ]:
import csv
import psycopg2
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import seaborn as sns
from scipy import stats
import networkx as nx
from statsmodels.tsa.seasonal import seasonal_decompose


db_host = "localhost"
db_port = "5432"
db_name = "lab10"
db_user = "kushhpatel"
db_password = ""

csv_file_path = 'world_development_data.csv'

conn = psycopg2.connect(
    host=db_host,
    port=db_port,
    database=db_name,
    user=db_user,
    password=db_password
)
cursor = conn.cursor()

def to_float(value):
    try:
        return float(value) if value != "" else None
    except ValueError:
        return None

with open(csv_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)

    for row in reader:
        year = row[0]  
        country_name = row[1]
        region = row[2]
        sub_region = row[3]
        interm_region = row[4]
        surf_area_sq_km = to_float(row[5])
        pop_dens = to_float(row[6])
        pop_growth = to_float(row[7])

        query = """
            INSERT INTO CountryInfo (Year, CountryName, Region, SubRegion, IntermRegion, SurfAreaSqKm, PopDens, PopGrowth)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(query, (year, country_name, region, sub_region, interm_region, surf_area_sq_km, pop_dens, pop_growth))

conn.commit()
cursor.close()
conn.close()

# -Code to insert data for 2-4 table


# Open the CSV file
with open(csv_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  # Skip the header row

    for row in reader:
        year, country_id = int(row[0]), int(row[1])
        # gdp, gdp_growth, adol_fert_rate, agri_val_add_gdp, dom_credit_gdp, exports_gdp, fert_rate, fdinet_bop, gni_cap_atlas, gni_atlas, gross_cap_form_gdp, imports_gdp, ind_val_add_gdp = map(to_float, row[2:15])
        # life_exp_birth, mort_rate_u5, net_migr, pop_total = map(to_float, row[2:6])

        # year, country_id, merch_trade_gdp = int(row[0]), int(row[1]), to_float(row[2])
        urban_pop_growth = to_float(row[2])

        # Check if CountryID and Year exist in CountryInfo
        if exists_in_country_info(country_id, year):
            # Insert data into EconomicIndicators
            # cursor.execute("""
            #     INSERT INTO EconomicIndicators (Year, CountryID, GDP, GDPGrowth, AdolFertRate, AgriValAddGDP, DomCreditGDP, ExportsGDP, FertRate, FDINetBoP, GNI_Cap_Atlas, GNI_Atlas, GrossCapFormGDP, ImportsGDP, IndValAddGDP)
            #     VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            # """, (year, country_id, gdp, gdp_growth, adol_fert_rate, agri_val_add_gdp, dom_credit_gdp, exports_gdp, fert_rate, fdinet_bop, gni_cap_atlas, gni_atlas, gross_cap_form_gdp, imports_gdp, ind_val_add_gdp))

            # cursor.execute("""
            #     INSERT INTO DemographicIndicators (Year, CountryID, LifeExpBirth, MortRateU5, NetMigr, PopTotal)
            #     VALUES (%s, %s, %s, %s, %s, %s)
            # """, (year, country_id, life_exp_birth, mort_rate_u5, net_migr, pop_total))

            # cursor.execute("""
            #     INSERT INTO TradeIndicators (Year, CountryID, MerchTradeGDP)
            #     VALUES (%s, %s, %s)
            # """, (year, country_id, merch_trade_gdp))
            cursor.execute("""
                INSERT INTO MiscellaneousIndicators (Year, CountryID, UrbanPopGrowth)
                VALUES (%s, %s, %s)
            """, (year, country_id, urban_pop_growth))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()


In [ ]:
# Write a Python script to visualize the GDP growth over the years for a specific country. Allow the user to input the country name, and plot a line graph representing the GDP gtowth.

def get_gdp_growth(db_params, country_name):
    
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cursor:
            
            query = """
            SELECT ei.Year, ei.GDPGrowth
            FROM EconomicIndicators ei
            JOIN CountryInfo ci ON ei.CountryID = ci.CountryID
            WHERE ci.CountryName = 'Afghanistan'
            ORDER BY ei.Year;
            """
            cursor.execute(query, (country_name,))
            return cursor.fetchall()

def plot_gdp_growth(data, country_name):
    if not data:
        print(f"No data available for {country_name}.")
        return

    years, growth = zip(*data)

    plt.figure(figsize=(10, 6))
    plt.plot(years, growth, marker='o')
    plt.title(f'GDP Growth Over the Years for {country_name}')
    plt.xlabel('Year')
    plt.ylabel('GDP Growth (%)')
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }

    country_name = input("Enter the country name to visualize GDP growth: ")
    gdp_growth_data = get_gdp_growth(db_params, country_name)
    plot_gdp_growth(gdp_growth_data, country_name)


In [ ]:
# 2. Investigate the correlation between the agricultural value added to GDP and fertility rate for a specific region. Create a scatter plot with agricultural value added on the x-axis and fertility rate on the y-axis

def get_agri_fertility_data(db_params, subregion):
    
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cursor:
            query = """
            SELECT ei.AgriValAddGDP, ei.FertRate
            FROM EconomicIndicators ei
            JOIN CountryInfo ci ON ei.CountryID = ci.CountryID
            WHERE ci.SubRegion = %s;
            """
            cursor.execute(query, (subregion,))
            return cursor.fetchall()

def plot_agri_fertility(data, subregion):
    if not data:
        print(f"No data available for the subregion: {subregion}.")
        return

    agri_val_add_gdp, fert_rate = zip(*data)
    plt.figure(figsize=(10, 6))
    plt.scatter(agri_val_add_gdp, fert_rate, alpha=0.7)
    plt.title(f'Agricultural Value Added vs Fertility Rate in {subregion}')
    plt.xlabel('Agricultural Value Added to GDP (%)')
    plt.ylabel('Fertility Rate')
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }

    subregion = input("Enter the subregion to analyze: ")
    data = get_agri_fertility_data(db_params, subregion)
    plot_agri_fertility(data, subregion)


In [ ]:
# 3. Use a geographical map library in Python to visualize the net migration rate for different countries. Apply color-coding to represent positive or negative net migration rates, and size of markers to represent the magnitude of the rate.

def get_migration_data(db_params):
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cursor:
            query = """
            SELECT ci.CountryName, di.NetMigr
            FROM DemographicIndicators di
            JOIN CountryInfo ci ON di.CountryID = ci.CountryID;
            """
            cursor.execute(query)
            return cursor.fetchall()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }

    migration_data = get_migration_data(db_params)
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    migration_df = pd.DataFrame(migration_data, columns=['Country', 'NetMigration'])
    world_migration = world.merge(migration_df, left_on='name', right_on='Country', how='left')
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    world_migration.plot(column='NetMigration', ax=ax, legend=True,
                         legend_kwds={'label': "Net Migration Rate",
                                      'orientation': "horizontal"},
                         cmap='coolwarm', missing_kwds={
                             'color': 'lightgrey',
                             'edgecolor': 'red',
                             'hatch': '///',
                             'label': 'Missing values'})
    plt.show()



In [ ]:
# 4. Visualize the distribution of urban population growth for a specific year using a histogram. Allow the user to input the year, and plot a histogram showing the frequency distribution of urban population growth rates.

def get_urban_growth_data(db_params, year):
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cursor:
            query = """
            SELECT mi.UrbanPopGrowth
            FROM MiscellaneousIndicators mi
            JOIN CountryInfo ci ON mi.CountryID = ci.CountryID
            WHERE mi.Year = %s;
            """
            cursor.execute(query, (year,))
            return [row[0] for row in cursor.fetchall() if row[0] is not None]

def plot_urban_growth_histogram(data, year):
    if not data:
        print(f"No data available for the year {year}.")
        return
    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=20, edgecolor='black')
    plt.title(f'Urban Population Growth Distribution in {year}')
    plt.xlabel('Urban Population Growth (%)')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }
    year = input("Enter the year to visualize urban population growth: ")

    try:
        year = int(year)
    except ValueError:
        print("Please enter a valid year.")
    else:
        urban_growth_data = get_urban_growth_data(db_params, year)
        plot_urban_growth_histogram(urban_growth_data, year)



In [ ]:
# 5. Generate a heatmap to visualize the correlation matrix between different economic indicators. Use the "EconomicIndicators" table and display the correlations between GDP, agricultural value added to GDP, and credit to GDP.

def get_economic_data(db_params):
    with psycopg2.connect(**db_params) as conn:
        query = """
        SELECT GDP, AgriValAddGDP, DomCreditGDP
        FROM EconomicIndicators;
        """
        return pd.read_sql_query(query, conn)

def plot_correlation_heatmap(df):
    correlation_matrix = df.corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
    plt.title('Correlation Matrix of Economic Indicators')
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }
    economic_data = get_economic_data(db_params)
    plot_correlation_heatmap(economic_data)



In [ ]:
# 6. Create a pie chart to represent the contribution of imports to GDP for a selected year. Allow the user to input the year, and plot a pie chart showing the percentage contribution of imports to the GDP of different countries.

def get_imports_data(db_params, year):
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cursor:
            query = """
            SELECT ci.CountryName, ei.ImportsGDP
            FROM EconomicIndicators ei
            JOIN CountryInfo ci ON ei.CountryID = ci.CountryID
            WHERE ei.Year = %s AND ei.ImportsGDP IS NOT NULL;
            """
            cursor.execute(query, (year,))
            return cursor.fetchall()

def plot_imports_pie_chart(data, year):
    if not data:
        print(f"No data available for the year {year}.")
        return

    countries, imports_gdp = zip(*data)

    plt.figure(figsize=(10, 8))
    plt.pie(imports_gdp, labels=countries, autopct='%1.1f%%', startangle=140)
    plt.title(f'Contribution of Imports to GDP in {year}')
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }
    year = input("Enter the year to visualize contribution of imports to GDP: ")

    try:
        year = int(year)
    except ValueError:
        print("Please enter a valid year.")
    else:
        imports_data = get_imports_data(db_params, year)
        plot_imports_pie_chart(imports_data, year)




In [ ]:
# 8. Create a box plot to visualize the distribution of mortality rates for children under 5 years old across different regions. Each box should represent a region, and the boxes should be ordered based on the median mortality rate.

def get_mortality_data(db_params):
    
    with psycopg2.connect(**db_params) as conn:
        query = """
        SELECT ci.Region, di.MortRateU5
        FROM DemographicIndicators di
        JOIN CountryInfo ci ON di.CountryID = ci.CountryID
        WHERE di.MortRateU5 IS NOT NULL;
        """
        return pd.read_sql_query(query, conn)

def plot_mortality_boxplot(df):
    ordered_regions = df.groupby('region')['mortrateu5'].median().sort_values().index

    plt.figure(figsize=(12, 8))
    sns.boxplot(x='mortrateu5', y='region', data=df, order=ordered_regions)
    plt.title('Distribution of Mortality Rates for Children Under 5 Across Regions')
    plt.xlabel('Mortality Rate Under 5 (per 1000 live births)')
    plt.ylabel('Region')
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }

    mortality_data = get_mortality_data(db_params)
    plot_mortality_boxplot(mortality_data)



In [ ]:
# 9. Generate a bubble chart to represent the relationship between GDP, life expectancy at birth, and fertility rate for countries in a specific sub-region. Use bubble size to indicate GDP, x-axis for life expectancy, and y-axis for fertility rate.

def get_country_data(db_params, sub_region):
    with psycopg2.connect(**db_params) as conn:
        query = """
        SELECT ci.CountryName, ei.GDP, ei.FertRate, di.LifeExpBirth
        FROM EconomicIndicators ei
        JOIN DemographicIndicators di ON ei.CountryID = di.CountryID
        JOIN CountryInfo ci ON ei.CountryID = ci.CountryID
        WHERE ci.SubRegion = %s;
        """
        return pd.read_sql_query(query, conn, params=[sub_region])

def plot_bubble_chart(df):
    df = df.dropna(subset=['gdp', 'fertrate', 'lifeexpbirth'])
    df['GDP_normalized'] = df['gdp'] / df['gdp'].max() * 1000

    plt.figure(figsize=(10, 6))
    plt.scatter(df['lifeexpbirth'], df['fertrate'], s=df['GDP_normalized'], alpha=0.5)
    plt.title(f'Relationship between GDP, Life Expectancy, and Fertility Rate in {sub_region}')
    plt.xlabel('Life Expectancy at Birth')
    plt.ylabel('Fertility Rate')
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }
    sub_region = input("Enter the sub-region to visualize: ")
    country_data = get_country_data(db_params, sub_region)
    plot_bubble_chart(country_data)




In [ ]:
# 10. Implement a choropleth map to visualize the average GDP growth for countries in different regions. Use a color scale to represent the magnitude of GDP growth, with darker shades indicating higher growth rates.

def get_gdp_growth_data(db_params):
    with psycopg2.connect(**db_params) as conn:
        query = """
        SELECT ci.Region, AVG(ei.GDPGrowth) as AvgGDPGrowth
        FROM EconomicIndicators ei
        JOIN CountryInfo ci ON ei.CountryID = ci.CountryID
        GROUP BY ci.Region;
        """
        return pd.read_sql_query(query, conn)

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }

    gdp_growth_data = get_gdp_growth_data(db_params)
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    world_gdp = world.merge(gdp_growth_data, left_on='continent', right_on='region', how='left')
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    world_gdp.plot(column='avggdpgrowth', ax=ax, legend=True,
                   legend_kwds={'label': "Average GDP Growth by Region",
                                'orientation': "horizontal"},
                   cmap='OrRd', missing_kwds={'color': 'lightgrey'})
    plt.title("Average GDP Growth for Countries in Different Regions")
    plt.show()


In [ ]:

# 11. Perform a linear regression analysis to predict GDP based on a selected economic indicator. Plot a scatter plot with the regression line and provide the equation of the line.

def get_gdp_and_credit_data(db_params):
    with psycopg2.connect(**db_params) as conn:
        query = """
        SELECT GDP, DomCreditGDP
        FROM EconomicIndicators
        WHERE GDP IS NOT NULL AND DomCreditGDP IS NOT NULL;
        """
        return pd.read_sql_query(query, conn)

def plot_regression_line(df):
    slope, intercept, r_value, p_value, std_err = stats.linregress(df['domcreditgdp'], df['gdp'])
    plt.figure(figsize=(10, 6))
    plt.scatter(df['domcreditgdp'], df['gdp'], alpha=0.5)
    line = slope * df['domcreditgdp'] + intercept
    plt.plot(df['domcreditgdp'], line, 'r', label=f'y = {slope:.2f}x + {intercept:.2f}')
    plt.title('Linear Regression Analysis: GDP vs Domestic Credit to GDP')
    plt.xlabel('Domestic Credit to GDP (%)')
    plt.ylabel('GDP')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }
    gdp_credit_data = get_gdp_and_credit_data(db_params)
    plot_regression_line(gdp_credit_data)



In [ ]:
# 12. Create a network graph to represent the trade relationships between countries. Nodes should be countries, and edges should represent trade connections. Use different edge weights to represent the strength of trade relationships.

def get_trade_data(db_params):
    with psycopg2.connect(**db_params) as conn:
        query = """
        SELECT ci.CountryName, ti.CountryID
        FROM TradeIndicators ti
        JOIN CountryInfo ci ON ti.CountryID = ci.CountryID;
        """
        return pd.read_sql_query(query, conn)

def create_trade_network(df):
    G = nx.Graph()
    for country in df['countryname'].unique():
        G.add_node(country)

    for _, row in df.iterrows():
        G.add_edge(row['countryname'], "Other Country")
    return G

def plot_network(G):
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, edge_color='gray')
    plt.title('Trade Relationships Between Countries')
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }
    trade_data = get_trade_data(db_params)
    trade_network = create_trade_network(trade_data)
    plot_network(trade_network)



In [ ]:
# 13. Apply time series decomposition to analyze the trend, seasonality, and residual components of the GDP growth for a specific country. Visualize each component separately.

def get_gdp_data(db_params, country_name):
    with psycopg2.connect(**db_params) as conn:
        query = """
        SELECT GDP
        FROM EconomicIndicators ei
        JOIN CountryInfo ci ON ei.CountryID = ci.CountryID
        WHERE ci.CountryName = %s
        ORDER BY ei.Year;
        """
        return pd.read_sql_query(query, conn, params=[country_name])

def decompose_gdp(df):
    decomposition = seasonal_decompose(df['gdp'], model='additive', period=1)
    plt.figure(figsize=(14, 7))
    plt.subplot(411)
    plt.plot(decomposition.observed, label='Observed')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(decomposition.trend, label='Trend')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(decomposition.seasonal,label='Seasonality')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(decomposition.resid, label='Residual')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }

    country_name = input("Enter the country name for GDP analysis: ")
    gdp_data = get_gdp_data(db_params, country_name)
    decompose_gdp(gdp_data)



In [ ]:
# 14. Generate a grouped bar chart to compare the fertility rates of countries in different sub-regions for a specific year. Each group should represent a sub-region, and each bar within a group should represent a country.

def get_fertility_data(db_params, year):
    with psycopg2.connect(**db_params) as conn:
        query = """
        SELECT ci.CountryName, ci.SubRegion, di.FertRate
        FROM EconomicIndicators di
        JOIN CountryInfo ci ON di.CountryID = ci.CountryID
        WHERE di.Year = 1999 AND di.FertRate IS NOT NULL;
        """
        return pd.read_sql_query(query, conn, params=[year])

def plot_grouped_bar_chart(df):
    pivoted_df = df.pivot(index='subregion', columns='countryname', values='fertrate')
    pivoted_df.plot(kind='bar', figsize=(12, 8), edgecolor='black')
    plt.title(f'Fertility Rates by Country in Different Sub-Regions for {year}')
    plt.xlabel('Sub-Region')
    plt.ylabel('Fertility Rate')
    plt.xticks(rotation=45)
    plt.grid(axis='y')
    plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }
    year = input("Enter the year for fertility rate comparison: ")

    try:
        year = int(year)
    except ValueError:
        print("Please enter a valid year.")
    else:
        fertility_data = get_fertility_data(db_params, year)
        plot_grouped_bar_chart(fertility_data)



In [ ]:
# 15. Visualize the proportion of the population living in urban areas over the years for a specific region. Use an area chart to represent the changing proportions.

def get_urban_pop_growth_data(db_params, region):
    with psycopg2.connect(**db_params) as conn:
        query = """
        SELECT mi.Year, AVG(mi.UrbanPopGrowth) as AvgUrbanPopGrowth
        FROM MiscellaneousIndicators mi
        JOIN CountryInfo ci ON mi.CountryID = ci.CountryID
        WHERE ci.Region = %s
        GROUP BY mi.Year
        ORDER BY mi.Year;
        """
        return pd.read_sql_query(query, conn, params=[region])

def plot_area_chart(df, region):
    if df.empty:
        print(f"No data available for the region: {region}.")
        return

    plt.figure(figsize=(12, 6))
    plt.fill_between(df['year'], df['avgurbanpopgrowth'], color="skyblue", alpha=0.4)
    plt.plot(df['year'], df['avgurbanpopgrowth'], color="Slateblue", alpha=0.6)
    plt.title(f'Average Urban Population Growth in {region} Over the Years')
    plt.xlabel('Year')
    plt.ylabel('Average Urban Population Growth (%)')
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    db_params = {
        "host": "localhost",
        "port": "5432",
        "database": "lab10",
        "user": "kushhpatel",
        "password": ""
    }
    region = input("Enter the region for urban population growth analysis: ")
    urban_pop_growth_data = get_urban_pop_growth_data(db_params, region)
    plot_area_chart(urban_pop_growth_data, region)
